In [43]:
import keras
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split

In [44]:
train_df = pd.read_csv('fashion_mnist/fashion-mnist_train.csv',sep=',')
test_df = pd.read_csv('fashion_mnist/fashion-mnist_test.csv', sep=',')
train_data = np.array(train_df, dtype = 'float32')
test_data = np.array(test_df, dtype='float32')
x_train = train_data[:50,1:]/255
y_train = train_data[:50,0]
x_test= test_data[:15,1:]/255
y_test=test_data[:15,0] 

In [45]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)
image_rows = 28
image_cols = 28
batch_size = 512
image_shape = (image_rows,image_cols,1)

In [46]:
x_train = x_train.reshape(x_train.shape[0],*image_shape)
x_test = x_test.reshape(x_test.shape[0],*image_shape)
x_validate = x_validate.reshape(x_validate.shape[0],*image_shape)

print("x_train shape = {}".format(x_train.shape))
print("x_test shape = {}".format(x_test.shape))
print("x_validate shape = {}".format(x_validate.shape))

x_train shape = (40, 28, 28, 1)
x_test shape = (15, 28, 28, 1)
x_validate shape = (10, 28, 28, 1)


In [47]:
name='3_layer'
cnn_model_3 = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', 
           input_shape=image_shape, kernel_initializer='he_normal', name='Conv2D-1'),
    MaxPooling2D(pool_size=2, name='MaxPool'),
    Dropout(0.25, name='Dropout-1'),
    Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
    Dropout(0.25, name='Dropout-2'),
    Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3'),
    Dropout(0.4, name='Dropout-3'),
    Flatten(name='flatten'),
    Dense(128, activation='relu', name='Dense'),
    Dropout(0.4, name='Dropout'),
    Dense(10, activation='softmax', name='Output')
], name=name)

In [48]:
cnn_model_3.summary()

Model: "3_layer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D-1 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 MaxPool (MaxPooling2D)      (None, 13, 13, 32)        0         
                                                                 
 Dropout-1 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 Conv2D-2 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 Dropout-2 (Dropout)         (None, 11, 11, 64)        0         
                                                                 
 Conv2D-3 (Conv2D)           (None, 9, 9, 128)         73856     
                                                                 
 Dropout-3 (Dropout)         (None, 9, 9, 128)         0   

In [49]:
cnn_model_3.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

history = cnn_model_3.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=50, verbose=1,
    validation_data=(x_validate, y_validate)
)

Epoch 1/50


2023-02-07 15:39:05.052923: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape in3_layer/Dropout-1/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-02-07 15:39:10.834111: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700
2023-02-07 15:39:16.594966: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-07 15:39:17.260352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1/1 [==============================] - 14s 14s/step - loss: 2.6154 - accuracy: 0.1000 - val_loss: 2.9200 - val_accuracy: 0.2000
Epoch 2/50
1/1 [==============================] - 0s 33ms/step - loss: 3.1648 - accuracy: 0.1250 - val_loss: 2.6943 - val_accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 36ms/step - loss: 2.1835 - accuracy: 0.1750 - val_loss: 2.4517 - val_accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 37ms/step - loss: 2.2121 - accuracy: 0.2000 - val_loss: 2.2578 - val_accuracy: 0.1000
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 2.2226 - accuracy: 0.1750 - val_loss: 2.2048 - val_accuracy: 0.3000
Epoch 6/50
1/1 [==============================] - 0s 31ms/step - loss: 2.0144 - accuracy: 0.2750 - val_loss: 2.2078 - val_accuracy: 0.3000
Epoch 7/50
1/1 [==============================] - 0s 33ms/step - loss: 2.0042 - accuracy: 0.3500 - val_loss: 2.2059 - val_accuracy: 0.3000
Epoch 8/50
1/1 [==============

In [56]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [60]:
%lprun -f cnn_model_3.fit cnn_model_3.fit( x_train, y_train, batch_size=batch_size, epochs=50, verbose=1, validation_data=(x_validate, y_validate) )

Epoch 1/50
1/1 [==============================] - ETA: 0s - loss: 0.2141 - accuracy: 0.9250

/home/fonzee/grad_school/profiling_1/venv/lib/python3.10/site-packages/line_profiler/ipython_extension.py:71: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding fit.__wrapped__ instead.
  profile = LineProfiler(*funcs)


1/1 [==============================] - 0s 157ms/step - loss: 0.2141 - accuracy: 0.9250 - val_loss: 2.4495 - val_accuracy: 0.4000
Epoch 2/50
1/1 [==============================] - 0s 96ms/step - loss: 0.1213 - accuracy: 0.9750 - val_loss: 2.4355 - val_accuracy: 0.3000
Epoch 3/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2195 - accuracy: 0.9000 - val_loss: 2.3912 - val_accuracy: 0.3000
Epoch 4/50
1/1 [==============================] - 0s 34ms/step - loss: 0.1169 - accuracy: 0.9750 - val_loss: 2.3770 - val_accuracy: 0.4000
Epoch 5/50
1/1 [==============================] - 0s 32ms/step - loss: 0.2473 - accuracy: 0.9250 - val_loss: 2.5135 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 30ms/step - loss: 0.1427 - accuracy: 0.9750 - val_loss: 2.6957 - val_accuracy: 0.4000
Epoch 7/50
1/1 [==============================] - 0s 30ms/step - loss: 0.1060 - accuracy: 1.0000 - val_loss: 2.8967 - val_accuracy: 0.3000
Epoch 8/50
1/1 [=====================

Timer unit: 1e-09 s

Total time: 1.12524 s
File: /home/fonzee/grad_school/profiling_1/venv/lib/python3.10/site-packages/keras/utils/traceback_utils.py
Function: error_handler at line 59

Line #      Hits         Time  Per Hit   % Time  Line Contents
    59                                               def error_handler(*args, **kwargs):
    60        51     200100.0   3923.5      0.0          if not tf.debugging.is_traceback_filtering_enabled():
    61                                                       return fn(*args, **kwargs)
    62                                           
    63        51      15100.0    296.1      0.0          filtered_tb = None
    64        51       7500.0    147.1      0.0          try:
    65        50 1124993201.0 22499864.0    100.0              return fn(*args, **kwargs)
    66                                                   except Exception as e:
    67                                                       filtered_tb = _process_traceback_frames(e._

In [61]:
%lprun -f cnn_model_3.predict cnn_model_3.predict(x_test)

1/1 [==============================] - 0s 16ms/step


/home/fonzee/grad_school/profiling_1/venv/lib/python3.10/site-packages/line_profiler/ipython_extension.py:71: UserWarning: Adding a function with a __wrapped__ attribute. You may want to profile the wrapped function by adding predict.__wrapped__ instead.
  profile = LineProfiler(*funcs)


Timer unit: 1e-09 s

Total time: 0.0524411 s
File: /home/fonzee/grad_school/profiling_1/venv/lib/python3.10/site-packages/keras/utils/traceback_utils.py
Function: error_handler at line 59

Line #      Hits         Time  Per Hit   % Time  Line Contents
    59                                               def error_handler(*args, **kwargs):
    60         1       6800.0   6800.0      0.0          if not tf.debugging.is_traceback_filtering_enabled():
    61                                                       return fn(*args, **kwargs)
    62                                           
    63         1        100.0    100.0      0.0          filtered_tb = None
    64         1        100.0    100.0      0.0          try:
    65         1   52433600.0 52433600.0    100.0              return fn(*args, **kwargs)
    66                                                   except Exception as e:
    67                                                       filtered_tb = _process_traceback_frames(e

In [64]:
%timeit Conv2D(32, kernel_size=3, activation='relu', input_shape=image_shape, kernel_initializer='he_normal', name='Conv2D-1')
%timeit MaxPooling2D(pool_size=2, name='MaxPool')
%timeit Dropout(0.25, name='Dropout-1')
%timeit Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2')
%timeit Dropout(0.25, name='Dropout-2')
%timeit Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3')
%timeit Dropout(0.4, name='Dropout-3')
%timeit Flatten(name='flatten')
%timeit Dense(128, activation='relu', name='Dense')
%timeit Dropout(0.4, name='Dropout')
%timeit Dense(10, activation='softmax', name='Output')

584 µs ± 853 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
275 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
304 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
585 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
302 µs ± 3.26 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
580 µs ± 6.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
305 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
212 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
405 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
300 µs ± 890 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
404 µs ± 721 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [66]:
%load_ext memory_profiler

In [67]:
%memit Conv2D(32, kernel_size=3, activation='relu', input_shape=image_shape, kernel_initializer='he_normal', name='Conv2D-1')
%memit MaxPooling2D(pool_size=2, name='MaxPool')
%memit Dropout(0.25, name='Dropout-1')
%memit Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2')
%memit Dropout(0.25, name='Dropout-2')
%memit Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3')
%memit Dropout(0.4, name='Dropout-3')
%memit Flatten(name='flatten')
%memit Dense(128, activation='relu', name='Dense')
%memit Dropout(0.4, name='Dropout')
%memit Dense(10, activation='softmax', name='Output')

peak memory: 4472.33 MiB, increment: 0.00 MiB
peak memory: 4435.11 MiB, increment: 0.00 MiB
peak memory: 4435.11 MiB, increment: 0.00 MiB
peak memory: 4435.32 MiB, increment: 0.00 MiB
peak memory: 4435.32 MiB, increment: 0.00 MiB
peak memory: 4435.32 MiB, increment: 0.00 MiB
peak memory: 4435.32 MiB, increment: 0.00 MiB
peak memory: 4435.32 MiB, increment: 0.00 MiB
peak memory: 4435.36 MiB, increment: 0.04 MiB
peak memory: 4435.36 MiB, increment: 0.00 MiB
peak memory: 4435.36 MiB, increment: 0.00 MiB
